In [2]:
from datasets import Dataset
import pandas as pd


balanced_df = pd.read_csv(r"../data/balanced_amazon_reviews.csv")

from sklearn.model_selection import train_test_split
from datasets import Dataset

# Step 1: Split in pandas
train_df, temp_df = train_test_split(balanced_df, test_size=0.2, stratify=balanced_df['label'], random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df['label'], random_state=42)

# Step 2: Convert to HuggingFace Dataset
train_ds = Dataset.from_pandas(train_df.reset_index(drop=True))
val_ds = Dataset.from_pandas(val_df.reset_index(drop=True))
test_ds = Dataset.from_pandas(test_df.reset_index(drop=True))


d:\InfoShare\Projects\sentiment-analysis\bert-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
train_df

,label,title,content
162291,0,"Looks great, works great...if you dont need a ...","PROS: Looks very cool, different mounting opti..."
189276,0,The Body Book,This definitely is geared for the upper end of...
10394,1,A vampire tale you can really bite into,"Sixteen-year-old Raven, an outcast who always ..."
192885,0,THIS IS ONE OF THE HARDEST AND WORST READS EVE...,If you think you'll be reading about Michael H...
146626,1,Awesome,I've seen the series before but didn't own it....
...,...,...,...
58292,1,Classic,Although I haven't read the book I felt this w...
161357,0,something missing,I bought this video for a film fest because Ca...
11688,1,Looks great,I used this product on some areas of a natural...
59296,0,waste of plastic and metal,This shaver is totally worthless. It gives an ...


In [4]:
from transformers import AutoTokenizer

# Load DistilBERT tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Tokenization function
def tokenize_function(example):
    return tokenizer(
        example["content"],        # This is the column with review text
        padding="max_length", 
        truncation=True,
        max_length=256
    )

# Apply tokenization to all datasets
train_ds = train_ds.map(tokenize_function, batched=True)
val_ds = val_ds.map(tokenize_function, batched=True)
test_ds = test_ds.map(tokenize_function, batched=True)


d:\InfoShare\Projects\sentiment-analysis\bert-env\lib\site-packages\transformers\utils\generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
d:\InfoShare\Projects\sentiment-analysis\bert-env\lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Map: 100%|██████████| 20000/20000 [00:01<00:00, 11282.54 examples/s]


In [5]:
def format_dataset(example):
    return {
        "input_ids": example["input_ids"],
        "attention_mask": example["attention_mask"],
        "labels": example["label"]  # 🔥 This is the fix
    }

train_ds = train_ds.map(format_dataset)
val_ds = val_ds.map(format_dataset)
test_ds = test_ds.map(format_dataset)


Map: 100%|██████████| 20000/20000 [00:04<00:00, 4331.42 examples/s]


In [6]:
# Remove unnecessary columns
train_ds = train_ds.remove_columns(["title", "content", "label"])
val_ds = val_ds.remove_columns(["title", "content", "label"])
test_ds = test_ds.remove_columns(["title", "content", "label"])

# Set format for PyTorch
train_ds.set_format("torch")
val_ds.set_format("torch")
test_ds.set_format("torch")


In [7]:
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Load pretrained model with 2 output labels (positive / negative)
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

# Define evaluation metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}


W0712 19:24:03.576017 29200 Lib\site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
d:\InfoShare\Projects\sentiment-analysis\bert-env\lib\site-packages\transformers\utils\generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
d:\InfoShare\Projects\sentiment-analysis\bert-env\lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'pre_classifier.bias', 'classifier.bias'

In [9]:

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    logging_dir="./logs",        
    logging_steps=50
)


In [10]:
from transformers import Trainer
from transformers import EarlyStoppingCallback


trainer = Trainer(
    model=model,                        # Your DistilBERT model
    args=training_args,                 # Training config
    train_dataset=train_ds,             # Your tokenized train dataset
    eval_dataset=val_ds,               # Your tokenized validation dataset
    tokenizer=tokenizer,               # The tokenizer you used earlier
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],  # 🔥 Early stopping here

    compute_metrics=compute_metrics    # The metrics function you defined
)

trainer.train()



  0%|          | 50/15000 [00:21<1:46:26,  2.34it/s]

{'loss': 0.4139, 'learning_rate': 4.9833333333333336e-05, 'epoch': 0.01}


  1%|          | 100/15000 [00:43<1:48:39,  2.29it/s]

{'loss': 0.2793, 'learning_rate': 4.966666666666667e-05, 'epoch': 0.02}


  1%|          | 150/15000 [01:15<5:21:30,  1.30s/it]

...

                                                        
100%|██████████| 15000/15000 [14:18:46<00:00,  1.19it/s]

{'eval_loss': 0.21949924528598785, 'eval_accuracy': 0.94675, 'eval_precision': 0.9467946794679468, 'eval_recall': 0.9467, 'eval_f1': 0.9467473373668683, 'eval_runtime': 3054.4879, 'eval_samples_per_second': 6.548, 'eval_steps_per_second': 0.102, 'epoch': 3.0}


100%|██████████| 15000/15000 [14:18:49<00:00,  3.44s/it]

{'train_runtime': 51529.4663, 'train_samples_per_second': 9.315, 'train_steps_per_second': 0.291, 'train_loss': 0.11396193806727727, 'epoch': 3.0}


TrainOutput(global_step=15000, training_loss=0.11396193806727727, metrics={'train_runtime': 51529.4663, 'train_samples_per_second': 9.315, 'train_steps_per_second': 0.291, 'train_loss': 0.11396193806727727, 'epoch': 3.0})

In [8]:
import accelerate
print(accelerate.__version__)


0.24.1


In [ ]:
import transformers
print(transformers.__version__)


d:\InfoShare\Projects\sentiment-analysis\bert-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


4.35.2


d:\InfoShare\Projects\sentiment-analysis\bert-env\lib\site-packages\transformers\utils\generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [1]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")


True
1
NVIDIA GeForce RTX 4060 Laptop GPU


In [11]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("./results/checkpoint-15000")
tokenizer = AutoTokenizer.from_pretrained("./results/checkpoint-15000")


In [12]:
model.save_pretrained("../models/distilbert-finetuned")
tokenizer.save_pretrained("../models/distilbert-finetuned")


('../models/distilbert-finetuned\\tokenizer_config.json',
 '../models/distilbert-finetuned\\special_tokens_map.json',
 '../models/distilbert-finetuned\\vocab.txt',
 '../models/distilbert-finetuned\\added_tokens.json',
 '../models/distilbert-finetuned\\tokenizer.json')

In [13]:
trainer.evaluate(test_ds)

100%|██████████| 313/313 [03:39<00:00,  1.43it/s]


{'eval_loss': 0.2329614758491516,
 'eval_accuracy': 0.94315,
 'eval_precision': 0.9422213351960882,
 'eval_recall': 0.9442,
 'eval_f1': 0.943209629888617,
 'eval_runtime': 220.8902,
 'eval_samples_per_second': 90.543,
 'eval_steps_per_second': 1.417,
 'epoch': 3.0}